# Part 1 : Data Preparation

# Data Import 

In [217]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData, Date
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [218]:
customer = pd.read_csv('customers.csv')
customer.head()

,customer_id,name,email
0,8,customer three,customer3@gmail.com
1,9,Shafran Naizer,mshafran13@gmail.com
2,10,Isuri Liyanage,liyanage.isukavi@gmail.com
3,12,Shafran Naizer,shafran@ieee.org
4,13,sss ss,ss@ieee.org


In [219]:
order = pd.read_csv('order.csv')
order.head()

,id,display_order_id,total_amount,created_at,customer_id
0,13392,YTFA,425,2024-10-14 15:12:43,1251.0
1,13393,N1U7,1650,2024-10-14 15:17:25,1251.0
2,13394,PADV,1365,2024-10-14 17:02:16,468.0
3,13395,OKVW,525,2024-10-14 17:03:36,1251.0
4,13396,2G3Q,525,2024-10-14 17:04:49,468.0


In [220]:
customer.isnull().sum()

customer_id     0
name            6
email          18
dtype: int64

In [221]:
order.isnull().sum()

id                    0
display_order_id     36
total_amount          0
created_at            0
customer_id         125
dtype: int64

In [222]:
#fill null values in customer table 
customer['name'].fillna("unknown", inplace=True)
customer['email'].fillna("N/A", inplace=True)

C:\Users\Divanka\AppData\Local\Temp\ipykernel_10808\2064338821.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer['name'].fillna("unknown", inplace=True)
C:\Users\Divanka\AppData\Local\Temp\ipykernel_10808\2064338821.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [223]:
customer.isnull().sum()

customer_id    0
name           0
email          0
dtype: int64

In [224]:
#fill null values in order table 
order['customer_id'].fillna("N/A", inplace=True)
#only filled customer id column, because that feature should be included in database table(order)

C:\Users\Divanka\AppData\Local\Temp\ipykernel_10808\1233562920.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order['customer_id'].fillna("N/A", inplace=True)
C:\Users\Divanka\AppData\Local\Temp\ipykernel_10808\1233562920.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  order['customer_id'].fillna("N/A", inplace=True)


In [225]:
order.isnull().sum()

id                   0
display_order_id    36
total_amount         0
created_at           0
customer_id          0
dtype: int64

In [226]:
customer['email']

0                        customer3@gmail.com
1                       mshafran13@gmail.com
2                 liyanage.isukavi@gmail.com
3                           shafran@ieee.org
4                                ss@ieee.org
                        ...                 
1109    alex.johnson+1729082517724@gmail.com
1110                 dominguhewa98@gmail.com
1111           liyanage.isukavi+22@gmail.com
1112          red.test506+99654648@gmail.com
1113                 lpw.ratnayake@gmail.com
Name: email, Length: 1114, dtype: object

In [227]:
#remover irrelevant columns 
customers= customer.drop(columns=['email'])
customers

,customer_id,name
0,8,customer three
1,9,Shafran Naizer
2,10,Isuri Liyanage
3,12,Shafran Naizer
4,13,sss ss
...,...,...
1109,1270,Alex Johnson
1110,1271,Dinuka Kaveen Dominguhewa
1111,1272,K L
1112,1273,fdv dfvfd


In [228]:
#remove irrelevant columns in order table
orders= order.drop(columns=['display_order_id'])
orders

,id,total_amount,created_at,customer_id
0,13392,425,2024-10-14 15:12:43,1251.0
1,13393,1650,2024-10-14 15:17:25,1251.0
2,13394,1365,2024-10-14 17:02:16,468.0
3,13395,525,2024-10-14 17:03:36,1251.0
4,13396,525,2024-10-14 17:04:49,468.0
...,...,...,...,...
8112,814,450000,2022-03-03 06:24:59,58.0
8113,826,125000,2022-03-03 10:17:47,58.0
8114,830,260000,2022-03-03 11:47:15,58.0
8115,845,125000,2022-03-04 01:30:58,58.0


In [229]:
customers = customers.rename(columns={'name':'customer_name'})

In [230]:
customers.head()

,customer_id,customer_name
0,8,customer three
1,9,Shafran Naizer
2,10,Isuri Liyanage
3,12,Shafran Naizer
4,13,sss ss


In [231]:
orders.head()

,id,total_amount,created_at,customer_id
0,13392,425,2024-10-14 15:12:43,1251.0
1,13393,1650,2024-10-14 15:17:25,1251.0
2,13394,1365,2024-10-14 17:02:16,468.0
3,13395,525,2024-10-14 17:03:36,1251.0
4,13396,525,2024-10-14 17:04:49,468.0


In [232]:
orders=orders.rename(columns={'id':'order_id',
                             'created_at':'order_date'})
orders.head()

,order_id,total_amount,order_date,customer_id
0,13392,425,2024-10-14 15:12:43,1251.0
1,13393,1650,2024-10-14 15:17:25,1251.0
2,13394,1365,2024-10-14 17:02:16,468.0
3,13395,525,2024-10-14 17:03:36,1251.0
4,13396,525,2024-10-14 17:04:49,468.0


In [242]:
# Define your MySQL connection parameters
MYSQL_USER = 'root'
MYSQL_PASSWORD = ''
MYSQL_HOST = 'sql12.freesqldatabase.com'
MYSQL_PORT = 3306
MYSQL_DATABASE = 'sql12740691'
Database_password: 'xnGri65wVa'

In [244]:
# # Establish connection
# connection = pymysql.connect(
#   host = MYSQL_HOST,
#   user = MYSQL_USER,
#   password = MYSQL_PASSWORD
# )

# # Create a cursor object to interact with the database
# cursor = connection.cursor()

In [245]:
# # Create a new database
# cursor.execute(f"CREATE DATABASE IF NOT EXISTS {MYSQL_DATABASE}")

1

In [248]:
# Create a MySQL database connection using SQLAlchemy
engine = create_engine(f'jdbc:mysql://sql12.freesqldatabase.com:3306/sql12740691')  # Replace with your MySQL details

# Import data into MySQL tables
customers.to_sql('customers', engine, if_exists='replace', index=False)  # Inserts customer data into the 'customers' table
orders.to_sql('orders', engine, if_exists='replace', index=False)  # Inserts order data into the 'orders' table

ArgumentError: Could not parse SQLAlchemy URL from string 'jdbc:mysql://sql12.freesqldatabase.com:3306/sql12740691'

In [237]:
try:
    # Create a connection to the MySQL database
    engine = create_engine('mysql+pymysql://root:@localhost/customer_order')
    connection = engine.connect()  # Establish the connection
    print("Connection successful!")
except SQLAlchemyError as e:
    print(f"Error connecting to the database: {e}")

Connection successful!
